### 1. 포트홀 데이터

In [ ]:
!pip install pandas openpyxl
!pip install geokakao

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import folium # 서울 지도 시각화
import geokakao as gk # 지오코딩
import requests # 역지오코딩
import pickle
import re

In [ ]:
### Loading the Data
pothole_path = "/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/서울시 포트홀 보수 위치 정보.xlsx"

# 포트홀 발생 위치
pothole = pd.read_excel(pothole_path, engine = "openpyxl")

In [ ]:
from google.colab import userdata
api = userdata.get('KAKAO_API_KEY')

In [ ]:
pothole.head()

,등록번호,처리상태,관할기관,행정구역,노선,방향,지번주소,도로명주소,파손종류
0,20240108165734001,보수,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),성북구 장위동 311-17,성북구 한천로101길 66,포트홀
1,20240108165727001,보수,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),성북구 장위동 301-18,성북구 한천로101길 66,포트홀
2,20240108165718001,보수,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),성북구 장위동 311-16,성북구 한천로101길 66,포트홀
3,20240108165711001,보수,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),NaN,성북구 한천로101길 66,포트홀
4,20240108165701001,보수,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),성북구 장위동 311-16,성북구 한천로101길 66,포트홀


In [ ]:
### 변수 값들 하나밖에 없음! 일단 제거하고 진행
print(pothole["처리상태"].unique())
print(pothole["파손종류"].unique())

['보수']
['포트홀']


In [ ]:
### 전처리
# 변수 두 개 제거
pothole_nona = pothole.drop(columns = ["처리상태", "파손종류"], axis = 1)
pothole_nona.head(3)
# "주소" 변수 만들기 - 지번주소 있으면 지번으로, 없으면 도로명
pothole_nona["주소"] = pothole["지번주소"].combine_first(pothole["도로명주소"])
pothole_nona = pothole_nona.dropna(subset = ["주소"])
pothole_nona.isna().sum()

,0
등록번호,0
관할기관,0
행정구역,18
노선,4107
방향,5524
지번주소,3817
도로명주소,2938
주소,0


In [ ]:
### "서울특별시" 붙이기
pothole_seoul = pothole_nona.copy()

pothole_seoul["서울_주소"] = pothole_seoul["주소"].apply(lambda x: "서울특별시 " + x if pd.notnull(x) and not x.startswith("서울") else x)

pothole_seoul.head(3)

,등록번호,관할기관,행정구역,노선,방향,지번주소,도로명주소,주소,서울_주소
0,20240108165734001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),성북구 장위동 311-17,성북구 한천로101길 66,성북구 장위동 311-17,서울특별시 성북구 장위동 311-17
1,20240108165727001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),성북구 장위동 301-18,성북구 한천로101길 66,성북구 장위동 301-18,서울특별시 성북구 장위동 301-18
2,20240108165718001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),성북구 장위동 311-16,성북구 한천로101길 66,성북구 장위동 311-16,서울특별시 성북구 장위동 311-16


In [ ]:
### 최종 데이터
pothole_final = pothole_seoul.drop(columns = ["지번주소", "도로명주소", "주소"])
pothole_final.rename(columns = {"행정구역" : "자치구"}, inplace = True)
pothole_final.head()

,등록번호,관할기관,자치구,노선,방향,서울_주소
0,20240108165734001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-17
1,20240108165727001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 301-18
2,20240108165718001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-16
3,20240108165711001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 한천로101길 66
4,20240108165701001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-16


#### 지오코딩 (위도, 경도 찾기)

In [ ]:
def convert_address_to_coordinates(address):
    url = f"https://dapi.kakao.com/v2/local/search/address.json"
    headers = {"Authorization": f"KakaoAK {api}"}
    params = {"query": address}

    try:
        response = requests.get(url, headers = headers, params = params)
        response.raise_for_status()
        result = response.json()

        if "documents" in result and len(result["documents"]) > 0:
            coordinates = result["documents"][0]["y"], result["documents"][0]["x"]
            return coordinates
        else:
            return "좌표를 못 찾았어요.."

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [ ]:
### 위도, 경도 찾기
# 21365개  - 2시간 반 정도
pothole_coor = pothole_final.copy()
gk.add_coordinates_to_dataframe(pothole_coor, "서울_주소")

In [ ]:
pothole_coor = pothole_coor.rename(columns = {"decimalLatitude" : "위도", "decimalLongitude" : "경도"})

pothole_coor.to_csv("/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/pothole_coordinates.csv")
pothole_coor.to_pickle("/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/pothole_coordinates.pickle")

In [ ]:
### 성능 확인
# NaN 있으면..
pothole_coor["위도"] = pothole_coor["위도"].replace(["None", None], np.nan)
pothole_coor["경도"] = pothole_coor["경도"].replace(["None", None], np.nan)

In [ ]:
pothole_coor.isna().sum()

,0
등록번호,0
관할기관,0
자치구,18
노선,4107
방향,5524
서울_주소,0
위도,228
경도,228


In [ ]:
# None 나오는 애들
pothole_coor[pothole_coor[["위도","경도"]].isna().all(axis = 1)]

,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도
29,20240108163446001,북부,종로구,온수~망우선,중랑구시계 (망우동산52-4),서울특별시 종로구 종로 86-3,NaN,NaN
252,20231229140406001,강서,구로구,독산~등촌선,등촌동로타리 (등촌동 505-23),서울특별시 구로구 구로중앙로 178-1,NaN,NaN
253,20231229140350001,강서,영등포구,NaN,NaN,서울특별시 영등포구 여의대로 60,NaN,NaN
304,20231228094152001,시설관리공단,동작구,올림픽대로,강동구 시계 (하일I.C),서울특별시 동작구 현충로 11,NaN,NaN
333,20231227170114001,남부,서초구,NaN,NaN,서울특별시 경기 과천시 주암동 산 2-12,NaN,NaN
...,...,...,...,...,...,...,...,...
22432,20230107080035001,강서,구로구,구로~염창선,올림픽대로유입램프 (염창동103),서울특별시 경기 광명시 철산동 460,NaN,NaN
22433,20230107080017001,강서,구로구,구로~염창선,올림픽대로유입램프 (염창동103),서울특별시 경기 광명시 철산동 460,NaN,NaN
22434,20230107075957001,강서,구로구,구로~염창선,올림픽대로유입램프 (염창동103),서울특별시 경기 광명시 철산동 460,NaN,NaN
22777,10000000519638030,성동,동대문구,온수~망우선,오류동시계 (온수동산17),서울 동대문구 휘경동 273,NaN,NaN


In [ ]:
# None 나오는 애들은 좌표를 못 찾아오네.. (228개 정도)
convert_address_to_coordinates("서울특별시 구로구 구로중앙로 178-1")

'좌표를 못 찾았어요..'

In [ ]:
### 일단은.. None 없애기
pothole_coor_nona = pothole_coor.dropna(subset = ["위도", "경도"])
pothole_coor_nona[["위도", "경도"]].isna( ).any()

,0
위도,False
경도,False


#### 역지오코딩 (행정동 찾기)

In [ ]:
### 역지오코딩
def convert_coordinates_to_address(latitude, longitude):
    y, x = str(latitude), str(longitude)
    url = f'https://dapi.kakao.com/v2/local/geo/coord2address.json?x={x}&y={y}'
    headers = {'Authorization': f'KakaoAK {api}'}

    try:
        response = requests.get(url, headers = headers)
        response.raise_for_status()
        result = response.json()

        if result.get("documents"):
            address_info = result["documents"][0].get("address")
            if address_info:
                return address_info.get("address_name")
            else:
                return "주소가 없어요.."

        else:
            return "주소가 없어요..2"

    except requests.exceptions.RequestException as e:
        print(f"Error: {e}")
        return None

In [ ]:
print(convert_coordinates_to_address(37.4981646510326, 127.028307900881))

서울 강남구 역삼동 858


In [ ]:
def get_dong(address):
    find = re.search(r"\b(\w+동)\b", address)
    if find:
        dong = find.group(1)
        return(dong)
    else:
        return(np.nan)

In [ ]:
def add_dong_to_dataframe(df, latitude_col, longitude_col):
    dongs = []
    loop = 0

    for lat, lon in zip(df[latitude_col], df[longitude_col]):
        address = convert_coordinates_to_address(lat, lon)

        if address:
            try:
                dong = get_dong(address)
            except:
                dong = np.nan
        else:
            dong = np.nan

        loop += 1

        dongs.append(dong)
        if loop % 1000 == 0:
            print(f"{loop}개 완료")

    df["행정동"] = dongs
    return df

In [ ]:
### 행정동 찾기
# 100개 - 28초
pothole_dong = pothole_coor_nona.copy()
add_dong_to_dataframe(pothole_dong,"위도","경도")

1000개 완료
2000개 완료
3000개 완료
4000개 완료
5000개 완료
6000개 완료
7000개 완료
8000개 완료
9000개 완료
10000개 완료
11000개 완료
12000개 완료
13000개 완료
14000개 완료
15000개 완료
16000개 완료
17000개 완료
18000개 완료
19000개 완료
20000개 완료
21000개 완료


,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도,행정동
0,20240108165734001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-17,37.6224340948957,127.046751013514,장위동
1,20240108165727001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 301-18,37.6225073242655,127.046803732217,장위동
2,20240108165718001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-16,37.6222914340147,127.046610690214,장위동
3,20240108165711001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 한천로101길 66,37.6219292248912,127.047320465849,장위동
4,20240108165701001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-16,37.6222914340147,127.046610690214,장위동
...,...,...,...,...,...,...,...,...,...
22979,10000000420888030,서부,서대문구,신월~신당선,신월I.C,서울 서대문구 합동 17-3,37.5614015932814,126.967056349082,합동
22980,10000000420585030,북부,종로구,NaN,NaN,서울 종로구 경운동 95,37.5763368263521,126.985164036382,안국동
22981,10000000419237030,서부,은평구,신림~진관내선,은평구시계 (진관내동569),서울 은평구 불광동 286-1,37.6153649381393,126.925379799666,불광동
22982,10000000418892030,서부,서대문구,신월~신당선,한양공고앞 (신당동251),서울 마포구 노고산동 31-11,37.554673736864,126.937561435278,노고산동


In [ ]:
pothole_dong.to_csv("/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/pothole_dongs.csv")
pothole_dong.to_pickle("/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/pothole_dongs.pickle")

In [ ]:
pothole_dongs[pothole_dongs["행정동"].isna()]

,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도,행정동
36,20240108162823001,북부,종로구,온수~망우선,오류동시계 (온수동산17),서울특별시 종로구 종로 243,37.5713165029695,127.004896238921,NaN
56,20240108131426001,북부,성북구,필동~도봉선,퇴계로3가로타리 (필동1가40),서울특별시 성북구 창경궁로35길 95,37.5881061830771,127.004425652365,NaN
62,20240108095623001,북부,종로구,양재~혜화선,혜화동로타리 (혜화동132),서울특별시 종로구 명륜2가 169-1,37.583778164056,126.999310626354,NaN
63,20240108095615001,북부,종로구,양재~혜화선,혜화동로타리 (혜화동132),서울특별시 종로구 명륜2가 169-1,37.583778164056,126.999310626354,NaN
83,20240105174814001,북부,성북구,NaN,NaN,서울특별시 성북구 보문로 195,37.591008815784,127.013813150499,NaN
...,...,...,...,...,...,...,...,...,...
22958,10000000431731030,서부,용산구,우면~중학선,우면동시계 (우면동418),서울 용산구 용산동2가 5-439,37.5441419447287,126.988282894924,NaN
22962,10000000428397030,북부,종로구,온수~망우선,오류동시계 (온수동산17),서울 종로구 종로4가 32-8,37.5707221670501,126.996534037355,NaN
22969,10000000423226030,북부,종로구,온수~망우선,중랑구시계 (망우동산52-4),서울 종로구 종로6가 94-1,37.5710303521384,127.008079544365,NaN
22978,10000000421472030,서부,용산구,노고산~창신선,창신동4거리 (창신동83),서울 용산구 용산동4가 1-1,37.5343252075797,126.987592344801,NaN


In [ ]:
## 이것도 그냥 못 찾아오는 거구나,,
get_dong("서울특별시 종로구 종로 243")

nan

#### 자치구 채우기

In [ ]:
### 경도, 위도 채워진 데이터
pothole_coor = pd.read_pickle("/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/pothole_coordinates.pickle")
print(f"총 {len(pothole_coor)}개의 데이터가 있습니다.")
pothole_coor.head(3)

총 21365개의 데이터가 있습니다.


,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도
0,20240108165734001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-17,37.6224340948957,127.046751013514
1,20240108165727001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 301-18,37.6225073242655,127.046803732217
2,20240108165718001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-16,37.6222914340147,127.046610690214


In [ ]:
### 행정동 채워진 데이터
pothole_dongs = pd.read_pickle("/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/pothole_dongs.pickle")
print(f"총 {len(pothole_dongs)}개의 데이터가 있습니다.")
pothole_dongs.head(3)

총 21137개의 데이터가 있습니다.


,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도,행정동
0,20240108165734001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-17,37.6224340948957,127.046751013514,장위동
1,20240108165727001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 301-18,37.6225073242655,127.046803732217,장위동
2,20240108165718001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-16,37.6222914340147,127.046610690214,장위동


In [ ]:
### 비어있는 데이터 확인
print(f"위도, 경도 없는 데이터 개수 : {pothole_coor['위도'].isna().sum().item()}")
print(f"행정동 없는 데이터 개수 : {pothole_dongs['행정동'].isna().sum().item()}")
print(f"자치구 없는 데이터 개수 : {pothole_dongs['자치구'].isna().sum().item()}")

위도, 경도 없는 데이터 개수 : 228
행정동 없는 데이터 개수 : 2583
자치구 없는 데이터 개수 : 18


In [ ]:
gu_na_indicies = pothole_dongs[pothole_dongs["자치구"].isna()].index
len(gu_na_indicies)

18

In [ ]:
nodong = pothole_dongs[pothole_dongs["행정동"].isna()]
len(nodong["서울_주소"].unique())

934

In [ ]:
### 자치구 채우기
def get_gu(address):
    find = re.search(r"\b(\w+구)\b", address)
    if find:
        gu = find.group(1)
        return(gu)
    else:
        return(np.nan)

def add_gu(df, indicies):
    for index in indicies:
        lat = df.loc[index, "위도"]
        lon = df.loc[index, "경도"]
        address = convert_coordinates_to_address(lat, lon)

        if address:
            try:
                gu = get_gu(address)
            except:
                gu = np.nan
        else:
            gu = np.nan

        df.loc[index, "자치구"] = gu
    return df

In [ ]:
pothole_final = add_gu(pothole_dongs, gu_na_indicies)

In [ ]:
pothole_final.isna().sum()

,0
등록번호,0
관할기관,0
자치구,0
노선,4058
방향,5458
서울_주소,0
위도,0
경도,0
행정동,2583


In [ ]:
pothole_final.to_pickle("/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/pothole_final.pickle")
pothole_final.to_csv("/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/pothole_dongs.csv")

In [ ]:
ppp = pd.read_pickle("/content/drive/MyDrive/2. KOREA UNIV./KUBIG/2025-2 Spring/분류 예측/DATA/pothole_final.pickle")
ppp.head()
ppp.isna().sum()

,0
등록번호,0
관할기관,0
자치구,0
노선,4058
방향,5458
서울_주소,0
위도,0
경도,0
행정동,2583


### 결측치 모아보기

In [ ]:
import pandas as pd
import pickle
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
pothole_final = pd.read_pickle("/content/drive/MyDrive/DATA_전처리/pothole_final2_idx.pickle")
pothole_final.isna().sum()

,0
등록번호,0
관할기관,0
자치구,0
노선,4097
방향,5514
서울_주소,0
위도,194
경도,194
행정동,523
idx,0


In [ ]:
pothole_final.head()

,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도,행정동,idx
0,20240108165734001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-17,37.6224340948957,127.046751013514,장위동,0
1,20240108165727001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 301-18,37.6225073242655,127.046803732217,장위동,1
2,20240108165718001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-16,37.6222914340147,127.046610690214,장위동,2
3,20240108165711001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 한천로101길 66,37.6219292248912,127.047320465849,장위동,3
4,20240108165701001,북부,성북구,미아~상계선,노원천주교회 (상계8동주공10단지),서울특별시 성북구 장위동 311-16,37.6222914340147,127.046610690214,장위동,4


In [ ]:
pothole_only_na = pothole_final[pothole_final[['위도','경도','행정동']].isna().any(axis = 1)]
len(pothole_only_na)

589

In [ ]:
pothole_only_na.head()

,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도,행정동,idx
28,20240108163446001,북부,종로구,온수~망우선,중랑구시계 (망우동산52-4),서울특별시 종로구 종로 86-3,None,None,NaN,28
35,20240108162823001,북부,종로구,온수~망우선,오류동시계 (온수동산17),서울특별시 종로구 종로 243,37.5713165029695,127.004896238921,NaN,35
55,20240108131426001,북부,성북구,필동~도봉선,퇴계로3가로타리 (필동1가40),서울특별시 성북구 창경궁로35길 95,37.5881061830771,127.004425652365,NaN,55
82,20240105174814001,북부,성북구,NaN,NaN,서울특별시 성북구 보문로 195,37.591008815784,127.013813150499,NaN,82
183,20231229141042001,강서,영등포구,궁동~영등포선,영등포로타리 (영등포동1가94),서울특별시 영등포구 영등포로53길 1,37.5187194385267,126.911881870096,NaN,183


In [ ]:
pothole_both_na = pothole_final[pothole_final[['위도','경도','행정동']].isna().all(axis = 1)]
print("둘 다 비어있는 거 :", len(pothole_both_na))
pothole_both_na.head()

둘 다 비어있는 거 : 128


,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도,행정동,idx
28,20240108163446001,북부,종로구,온수~망우선,중랑구시계 (망우동산52-4),서울특별시 종로구 종로 86-3,None,None,NaN,28
208,20231229140406001,강서,구로구,독산~등촌선,등촌동로타리 (등촌동 505-23),서울특별시 구로구 구로중앙로 178-1,None,None,NaN,208
209,20231229140350001,강서,영등포구,NaN,NaN,서울특별시 영등포구 여의대로 60,None,None,NaN,209
253,20231228094152001,시설관리공단,동작구,올림픽대로,강동구 시계 (하일I.C),서울특별시 동작구 현충로 11,None,None,NaN,253
399,20231226111146001,시설관리공단,영등포구,올림픽대로,강동구 시계 (하일I.C),서울특별시 강서구 양천로77길 32,None,None,NaN,399


In [ ]:
pothole_only_wg_na = pothole_final[pothole_final[['위도','경도']].isna().all(axis = 1) & pothole_final['행정동'].notna()]
print("위도경도만 없는 거 : ",len(pothole_only_wg_na))
pothole_only_wg_na.head()

위도경도만 없는 거 :  66


,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도,행정동,idx
482,20231225057116010,시설관리공단,강서구,올림픽대로,강동구 시계 (하일I.C),서울 강서구 방화동 11-2,None,None,방화동,482
1535,20231128210453001,성동,동대문구,창신~답십리선,NaN,서울특별시 로동대문구 전농동 103-156,None,None,전농동,1535
1541,20231128205620001,성동,성동구,내곡~월곡선,NaN,서울특별시 편성동구 행당동 192-40,None,None,행당동,1541
2260,20231031112839001,시설관리공단,강서구,올림픽대로,강동구 시계 (하일I.C),서울특별시 강서구 방화동 76,None,None,방화동,2260
3170,20230930204651001,성동,광진구,잠실철교선,NaN,서울특별시 장광진구 구의동 546-3,None,None,구의동,3170


In [ ]:
pothole_only_hang_na = pothole_final[pothole_final[['위도','경도']].notna().all(axis = 1) & pothole_final['행정동'].isna()]
print("행정동만 없는 거 : ",len(pothole_only_hang_na))
pothole_only_hang_na.head()

행정동만 없는 거 :  395


,등록번호,관할기관,자치구,노선,방향,서울_주소,위도,경도,행정동,idx
35,20240108162823001,북부,종로구,온수~망우선,오류동시계 (온수동산17),서울특별시 종로구 종로 243,37.5713165029695,127.004896238921,NaN,35
55,20240108131426001,북부,성북구,필동~도봉선,퇴계로3가로타리 (필동1가40),서울특별시 성북구 창경궁로35길 95,37.5881061830771,127.004425652365,NaN,55
82,20240105174814001,북부,성북구,NaN,NaN,서울특별시 성북구 보문로 195,37.591008815784,127.013813150499,NaN,82
183,20231229141042001,강서,영등포구,궁동~영등포선,영등포로타리 (영등포동1가94),서울특별시 영등포구 영등포로53길 1,37.5187194385267,126.911881870096,NaN,183
314,20231227090352001,시설관리공단,성동구,강변북로,광진구 광장동시계,서울특별시 성동구 둘레길 63-1,37.53624033553,127.050674400193,NaN,314


In [ ]:
# 위도경도, 행정동 모두 비어있음 - 128
pothole_both_na.to_csv("/content/drive/MyDrive/Pothole 결측치/both_na.csv", encoding='cp949')
# 위도경도만 비어 있음 - 66
pothole_only_wg_na.to_csv("/content/drive/MyDrive/Pothole 결측치/wg_na.csv", encoding='cp949')
# 행정동만 비어 있음 - 395
pothole_only_hang_na.to_csv("/content/drive/MyDrive/DATA_전처리/pothole.pickle", encoding='cp949')

NameError: name 'pothole_only_hang_na' is not defined